In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!rm -rf streamjoy_scratch

In [3]:
def get_data_aspect(ds):
    return ds.grid_yt.size/ds.grid_xt.size
    
def select_subset(ds, x,y, width, aspect):
    selection = {
        'grid_xt':slice(x, x+width),
        'grid_yt':slice(y, y+int(width*aspect)),
    }
    return ds.isel(**selection)

In [4]:
import matplotlib.colors as colors
oceanmap = plt.get_cmap('copper')(np.linspace(0, 1, 100))
oceanmap = oceanmap[:,(2,1,0,3)]
oceanmap = colors.ListedColormap(oceanmap)

landmap = plt.get_cmap('copper')(np.linspace(0, 1, 100))
landmap = landmap[:,(0,1,2,3)]
landmap = colors.ListedColormap(landmap)

# Create an array of RGBA values from white to transparent white
n_colors = 256
transparent = np.ones((n_colors, 4))  # all white, RGBA
transparent[:, 3] = np.linspace(1.0, 0.0, n_colors)  # alpha channel goes from 1 to 0

# transparent to white
transparent_to_white = colors.ListedColormap(np.flipud(transparent), name='transparent_to_white')

In [5]:
mask = xr.open_dataset("/home/aw2143/hackathon/landmask.nc")['HGTsfc']
mask.name = 'landmask'
# mask.plot()

In [6]:
ds = xr.open_dataset("/scratch/cimes/aw2143/hackathon/Ta_fullTile.zarr/",  engine='zarr', chunks={})
ds_cwp = xr.open_dataset("/scratch/cimes/aw2143/hackathon/cwp_fullTile.zarr/", engine='zarr', chunks={}).rename({'__xarray_dataarray_variable__':'cwp'})

ds = xr.merge([ds, ds_cwp, mask], join='inner')
ds_movie = ds

In [7]:
# factor = 6/16 # fill 2/3 of the height of a 16:9 screen
# y_start = 1200
# y_end = y_start+int(ds_movie.grid_xt.size*factor)
# ds_movie = ds_movie.isel(grid_yt=slice(y_start, y_end))

# data_aspect = get_data_aspect(ds_movie)
# print(data_aspect)
# ds_movie


factor = 9/16 # fill entire 16:9 screen
y_start = 1100
y_end = y_start+int(ds_movie.grid_xt.size*factor)
ds_movie = ds_movie.isel(grid_yt=slice(y_start, y_end))

data_aspect = get_data_aspect(ds_movie)
ds_movie

<xarray.Dataset> Size: 102GB
Dimensions:   (time: 2400, grid_yt: 1728, grid_xt: 3072)
Coordinates:
  * grid_xt   (grid_xt) float64 25kB 1.0 2.0 3.0 ... 3.071e+03 3.072e+03
  * grid_yt   (grid_yt) float64 14kB 1.101e+03 1.102e+03 ... 2.827e+03 2.828e+03
  * time      (time) object 19kB 2016-08-11 00:03:00 ... 2016-08-16 00:00:00
Data variables:
    Ta        (time, grid_yt, grid_xt) float32 51GB dask.array<chunksize=(1, 1728, 3072), meta=np.ndarray>
    cwp       (time, grid_yt, grid_xt) float32 51GB dask.array<chunksize=(1, 1728, 3072), meta=np.ndarray>
    landmask  (grid_yt, grid_xt) bool 5MB ...

In [8]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=32, threads_per_worker=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 500.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38305,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:46057,Total threads: 1
Dashboard: http://127.0.0.1:43477/status,Memory: 15.62 GiB
Nanny: tcp://127.0.0.1:33195,


In [9]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cftime

def draw_clock_inset(ax, time, size="15%", loc='upper right'):
    """
    Draws a clock inset on the given axis using a datetime-like object.
    
    Parameters:
        ax : matplotlib Axes
            The main axes to draw the inset into.
        time : datetime or cftime.datetime
            A datetime object (e.g., from xarray time coordinates).
        size : str or float
            Size of the inset (e.g., '25%' or float fraction).
        loc : str
            Location of the inset (e.g., 'upper right', 'lower left').
    """
    # Extract hour and minute
    hour = time.hour
    minute = time.minute

    # Determine day/night style
    is_day = (hour > 6) & (hour < 18)
    face_color = 'white' if is_day else 'black'
    hand_color = 'black' if is_day else 'white'
    tick_color = hand_color

    # Convert to 12-hour format angles
    hour12 = hour % 12
    hour_angle = np.deg2rad(90 - (30 * hour12 + 0.5 * minute))
    minute_angle = np.deg2rad(90 - 6 * minute)

    # Create inset axis
    ax_inset = inset_axes(ax, width=size, height=size, loc=loc)
    ax_inset.set_aspect('equal')
    ax_inset.axis('off')

    # Draw clock face
    ax_inset.add_artist(plt.Circle((0, 0), 1, facecolor=face_color, linewidth=2, edgecolor=tick_color))

    # Hour ticks
    for i in range(12):
        angle = np.deg2rad(90 - i * 30)
        x_outer, y_outer = np.cos(angle), np.sin(angle)
        x_inner, y_inner = 0.9 * x_outer, 0.9 * y_outer
        ax_inset.plot([x_inner, x_outer], [y_inner, y_outer], color=tick_color, lw=1)

    # Hour hand
    ax_inset.plot([0, 0.5 * np.cos(hour_angle)], [0, 0.5 * np.sin(hour_angle)], color=hand_color, lw=3)

    # Minute hand
    ax_inset.plot([0, 0.8 * np.cos(minute_angle)], [0, 0.8 * np.sin(minute_angle)], color=hand_color, lw=2)

    # Center dot
    ax_inset.plot(0, 0, 'o', color=hand_color)

    ax_inset.set_xlim(-1.1, 1.1)
    ax_inset.set_ylim(-1.1, 1.1)

In [ ]:
from streamjoy import stream, wrap_matplotlib
import numpy as np
import streamjoy
streamjoy.config["fps"]= 15

# set up figure size to match 16:9 screen
dpi = 600
total_width = 3840*2 #8k
x_len = total_width/dpi # inches
y_len = x_len / 16 * 9


clim = {
    'land':dict(vmin=290, vmax=311),
    'ocean':dict(vmin=294, vmax=301),
    'clouds':dict(vmax=2, vmin=0),
}


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

def make_cbar(fig, ax, artist, x, y, label):
    width = 0.05
    height = 0.05
    # grey = '0.3'
    grey = '0.0'

    # Create inset axis for the colorbar
    cax = ax.inset_axes([x, y, width, height])

    # Create a checkerboard pattern
    n = 20  # number of squares
    checker = np.indices((n, n)).sum(axis=0) % 2  # alternating 0 and 1

    # Show checker pattern
    cax.imshow(checker, cmap='Greys', interpolation='nearest',
               extent=[0, 1, 0, 1], aspect='auto', vmin=0, vmax=1, alpha=0.2, zorder=0)

    # Draw the colorbar on top
    cbar = fig.colorbar(artist, cax=cax, orientation='horizontal', aspect=1, shrink=0.5)
    cbar.ax.tick_params(labelsize=10, colors=grey)
    cbar.ax.set_title(label, fontsize=10, color=grey)

    # Set spine color
    for spine in cbar.ax.spines.values():
        spine.set_color(grey)

    return cax, cbar

@wrap_matplotlib()
def plot_func(da_time, ds_frame_other, **plot_kwargs):
    fig, ax_top = plt.subplots(
        figsize=(x_len, y_len),
        dpi=dpi
    )
    fig.patch.set_facecolor('black')
    

    # Plot the top frame with no borders
    land = ds_frame_other['Ta'].where(mask).plot(
        ax=ax_top, cmap=landmap, add_colorbar=False, add_labels=False, **clim['land'])
    ocean = ds_frame_other['Ta'].where(~mask).plot(
        ax=ax_top, cmap=oceanmap, add_colorbar=False, add_labels=False, **clim['ocean'])
    clouds = ds_frame_other.cwp.plot(
        ax=ax_top, cmap=transparent_to_white, add_colorbar=False, add_labels=False, **clim['clouds'])
    make_cbar(fig, ax_top, land, 0.7, 0.91, 'LAND')
    make_cbar(fig, ax_top, ocean, 0.775, 0.91, 'OCEAN')
    make_cbar(fig, ax_top, clouds, 0.85, 0.91, 'CLOUDS')
    
    ax_top.set_facecolor('black')
    ax_top.set_xticks([])
    ax_top.set_yticks([])
    ax_top.set_aspect(1)
    ax_top.axis('off')  # Turn off everything

    # Remove spines just in case
    for spine in ax_top.spines.values():
        spine.set_visible(False)

    ax_top.margins(0)

    # Choose a cftime datetime (e.g., the first one)
    selected_time = da_time.data.flatten()[0]
    
    # Draw clock for selected time
    draw_clock_inset(ax_top, selected_time, loc="upper left")

    fig.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0)

    return fig

# n_frames = 3 # testing
n_frames = ds_movie.time.size

st = stream(
    ds_movie, 
    dim='time', 
    var='time', # this natively only works with a dataarray ( I am only passing the time as a dummy here)
    renderer=plot_func, 
    renderer_iterables=[
        [ds_movie.isel(time=t) for t in range(n_frames)], # I am passing each timestep again as a daataset!!!
    ],
    max_frames=n_frames,
)
st.write(
    'single_panel_8k_.mp4',
    client = client
)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 500.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38305,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:46057,Total threads: 1
Dashboard: http://127.0.0.1:43477/status,Memory: 15.62 GiB
Nanny: tcp://127.0.0.1:33195,


/home/jbusecke/.conda/envs/easy25/lib/python3.12/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 14.69 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
